Before you run any of the code below, please make sure that you have installed the necessary packages using the line of code below

In [1]:
#May need to install programmes in command prompt first with:
# pip install pandas 
# pip install numpy 
# pip install dash 
# pip install scipy 
# pip install matplotlib
#To run in browser need to use this web address http://127.0.0.1:2234/ (2234 you need to take from the port number at the end of this block of code after you run it) this will vary for each line of code below 


In [2]:
# Install necessary packages if you haven't already
import pandas as pd
import numpy as np
from dash import Dash, html, dash_table, dcc, callback, Output, Input, State, ALL
import dash_daq as daq
import base64
import io
from scipy.interpolate import interp1d
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure

# Initialize the app
"""This line is known as the Dash constructor and it is responsible for 
initialising your app, its always the same for all Dash apps"""
app = Dash(__name__)

# Color and marker options
color_options = list(matplotlib.colors.cnames.keys())
marker_options = ['o', '^', 's', 'd', '+', 'x', '*']
font_options = ['Arial', 'Helvetica', 'Times New Roman', 'Courier New', 'Verdana', 'Georgia', 'Palatino', 'Garamond', 'Comic Sans MS', 'Impact']

# App layout
"""The app layout represents the app components that will be displayed in 
web browser and here it is provided as a dash component but could also be 
provided as a list. The components added are:
    html.Div = has properties children to add text content to the page
    dcc = a module (stands for Dash core components), give you acess to 
    interactive components 
    dcc.Upload = given an id name as this will be used by callback to identify 
    teh components 
    dcc.Dropdown = components taht ceeates customizable dropdown menus for 
    selecting one or multiple items from a list of options, multi propety 
    allows you to sletc more than one value at a time, option
    (what user sees) and value (what gest outputtes in callback) are the 
    first two argyments in dcc.dropdown
    placeholder allows you to deifne text when no value is selected
    dcc.Input =  creates input fields for text, numbers, passwords, and more, allowing dynamic user input.


     """
app.layout = html.Div([
    html.H1("Frequency Sweeps", style={'textAlign': 'center'}),
    dcc.Upload(
        id='upload-data',
        children=html.Button('Upload Excel File', style={'margin': '10px'}),
        multiple=False
    ),
    dcc.Dropdown(id='sheet-dropdown', placeholder="Select Sheet", style={'margin': '10px'}),
    dcc.Dropdown(id='test-dropdown', placeholder="Select Tests", multi=True, style={'margin': '10px'}),
    dcc.Dropdown(id='color-picker', options=[{'label': color, 'value': color} for color in color_options],
                 placeholder="Select Plot Color", multi=True, style={'margin': '10px'}),
    dcc.Dropdown(id='marker-picker', options=[{'label': marker, 'value': marker} for marker in marker_options],
                 placeholder="Select Interpolated Line Marker", multi=False, style={'margin': '10px'}),
    dcc.Dropdown(id='font-picker', options=[{'label': font, 'value': font} for font in font_options],
                 placeholder="Select Font Type", style={'margin': '10px'}),
    html.Div([
        dcc.Input(id='marker-size', type='number', placeholder='Marker Size', style={'margin': '10px'}),
    ]),
    html.Div([
        dcc.Input(id='x-min', type='number', placeholder='X-axis Min', style={'margin': '10px'}),
        dcc.Input(id='x-max', type='number', placeholder='X-axis Max', style={'margin': '10px'}),
        dcc.Input(id='interp-x-min', type='number', placeholder='Interpolation X-min', style={'margin': '10px'}),
        dcc.Input(id='interp-x-max', type='number', placeholder='Interpolation X-max', style={'margin': '10px'}),
        dcc.Checklist(
            id='gridlines-toggle',
            options=[{'label': 'Show Gridlines', 'value': 'show'}],
            value=['show'],
            style={'margin': '10px'}
        ),
        dcc.Checklist(
            id='find-g-equality',
            options=[{'label': 'Find G\' = G\" Point', 'value': 'find'}],
            value=[],
            style={'margin': '10px'}
        ),
        dcc.Checklist(
            id='plot-interpolated',
            options=[{'label': 'Plot Interpolated Lines', 'value': 'plot'}],
            value=[],
            style={'margin': '10px'}
        ),
    ]),
    html.Div(id='plot-labels-div'),
    html.Button('Add Plot Label', id='add-plot-label', n_clicks=0, style={'margin': '10px'}),
    html.Div([
        dcc.Input(id='font-size', type='number', placeholder='Font Size', style={'margin': '10px'}),
    ]),
    html.Div([
        dcc.Input(id='freq-specific', type='number', placeholder='Angular Frequency for Raw Data', style={'margin': '10px'}),
        dcc.Checklist(
            id='raw-data-toggle',
            options=[{'label': 'Get Raw Data at Specific Frequency', 'value': 'raw'}],
            value=[],
            style={'margin': '10px'}
        ),
    ]),
    dash_table.DataTable(id='data-table', page_size=10, style_table={'overflowX': 'auto'}),
    html.Div(id='raw-data-output', style={'margin': '10px'}),
    html.Div(id='plot-img'),
    html.Div(id='crossover-frequencies', style={'margin': '10px'}),
    html.Div(id='error-output', style={'color': 'red', 'margin': '10px'})
])

# Callback to add input fields for plot labels
"""To work with call back in a Dash app we use the callback module to 
allow user more freedom to interact with the app. Inputs and OUtputs are 
properties of the component that has the ID. The n_clicks property is 
an integer taht represents the number of times teh element has been 
clicked. You can use the n_clicks to trigger a callback and use teh value of 
n_clicks in your callback logic

By writing this decorator, we're telling Dash to call this function for us whenever the value of the "input" component (the text box) changes in order to update the children of the "output" component on the page (the HTML div).
  """
@callback(
    Output('plot-labels-div', 'children'),
    Input('add-plot-label', 'n_clicks'),
    State('plot-labels-div', 'children')
)

#Callback function to add plot labels 
def add_plot_labels(n_clicks, children):
    if children is None:
        children = []
    new_input = dcc.Input(id={'type': 'plot-label', 'index': n_clicks}, type='text', placeholder=f'Plot Label {n_clicks}', style={'margin': '10px'})
    children.append(new_input)
    return children

# Add controls to build the interaction
@callback(
    Output('sheet-dropdown', 'options'),
    Input('upload-data', 'contents'),
    State('upload-data', 'filename')
)
def update_sheet_dropdown(contents, filename):
    if contents is None:
        return []

    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    df = pd.ExcelFile(io.BytesIO(decoded))
    sheet_names = df.sheet_names
    return [{'label': sheet, 'value': sheet} for sheet in sheet_names]

@callback(
    Output('test-dropdown', 'options'),
    Input('sheet-dropdown', 'value'),
    State('upload-data', 'contents')
)
def update_test_dropdown(sheet_name, contents):
    if sheet_name is None or contents is None:
        return []

    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    df_tests = pd.read_excel(io.BytesIO(decoded), sheet_name=sheet_name, usecols=['Application:', 'Anton Paar RheoCompass™ V1.22.0.0'])
    df_tests['Application:'] = df_tests['Application:'].fillna('')
    df_tests = df_tests[df_tests['Application:'].str.startswith('Test:')]
    tests = df_tests['Anton Paar RheoCompass™ V1.22.0.0'].tolist()
    return [{'label': test, 'value': i} for i, test in enumerate(tests)]

@callback(
    Output('data-table', 'data'),
    Output('data-table', 'columns'),
    Output('plot-img', 'children'),
    Output('raw-data-output', 'children'),
    Output('crossover-frequencies', 'children'),
    Output('error-output', 'children'),  # New output for error messages
    Input('test-dropdown', 'value'),
    Input('color-picker', 'value'),
    Input('marker-picker', 'value'),
    Input('x-min', 'value'),
    Input('x-max', 'value'),
    Input('interp-x-min', 'value'),
    Input('interp-x-max', 'value'),
    Input('gridlines-toggle', 'value'),
    Input('find-g-equality', 'value'),
    Input('plot-interpolated', 'value'),
    Input({'type': 'plot-label', 'index': ALL}, 'value'),
    Input('font-size', 'value'),
    Input('font-picker', 'value'),
    Input('marker-size', 'value'),
    Input('freq-specific', 'value'),
    Input('raw-data-toggle', 'value'),
    State('sheet-dropdown', 'value'),
    State('upload-data', 'contents')
)
def update_output(selected_tests, selected_colors, marker, x_min, x_max, interp_x_min, interp_x_max, gridlines_toggle, find_g_equality, plot_interpolated, plot_labels, font_size, font_picker, marker_size, freq_specific, raw_data_toggle, sheet_name, contents):
    error_message = ""
    try:
        if not selected_tests or sheet_name is None or contents is None:
            return [], [], '', '', '', "No tests or data selected."

        if font_size is None:
            font_size = 12  # Set a default font size if None

        if font_picker is None:
            font_picker = 'Arial'  # Set a default font if None

        if marker_size is None:
            marker_size = 10  # Set a default marker size if None

        if selected_colors is None:
            selected_colors = color_options  # Set a default color list if None

        content_type, content_string = contents.split(',')
        decoded = base64.b64decode(content_string)
        df_data = pd.read_excel(io.BytesIO(decoded), sheet_name=sheet_name, usecols=[
            'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5','Unnamed: 6']).dropna(how='any')

        # Define the value to split the DataFrame on
        split_value = 'Storage Modulus'
       
        # Group the DataFrame by the value to split on
        grouped = df_data.groupby((df_data['Unnamed: 4'] == split_value).cumsum())
       
        # Create a list of the tables of data for each test
        df_list = [group for name, group in grouped if len(group) >= 3]

        # Ensure we have the same number of selected colors as tests
        selected_colors = selected_colors * (len(selected_tests) // len(selected_colors) + 1)

        # Prepare data and columns for the DataTable
        df_selected = pd.concat([df_list[i] for i in selected_tests])
        data = df_selected.to_dict('records')
        columns = [{'name': col, 'id': col} for col in df_selected.columns]

        # Plotting
        raw_data_output = []
        crossover_freq_output = []
        fig, ax = plt.subplots(figsize=(12,12))
       
        for i, test_index in enumerate(selected_tests):
            df_test = df_list[test_index]
            freq = df_test['Unnamed: 6'].iloc[2:].astype(float)
            storage_moduli = df_test['Unnamed: 4'].iloc[2:].astype(float)
            loss_moduli = df_test['Unnamed: 5'].iloc[2:].astype(float)

            # Interpolation range
            if interp_x_min is not None and interp_x_max is not None:
                mask = (freq >= interp_x_min) & (freq <= interp_x_max)
                freq_interp = freq[mask]
                storage_moduli_interp = storage_moduli[mask]
                loss_moduli_interp = loss_moduli[mask]

                # Interpolate data
                if len(freq_interp) > 1:  # Ensure there are enough points for interpolation
                    freq_new = np.logspace(np.log10(freq_interp.min()), np.log10(freq_interp.max()), num=500)
                    storage_moduli_interp = interp1d(freq_interp, storage_moduli_interp, kind='cubic')(freq_new)
                    loss_moduli_interp = interp1d(freq_interp, loss_moduli_interp, kind='cubic')(freq_new)
                else:
                    freq_new = np.array([])
                    storage_moduli_interp = np.array([])
                    loss_moduli_interp = np.array([])

            ax.plot(freq, storage_moduli, 'o-', label=f'{plot_labels[i] if i < len(plot_labels) else test_index} G\'', color=selected_colors[i])
            ax.plot(freq, loss_moduli, '^-', label=f'{plot_labels[i] if i < len(plot_labels) else test_index} G\"', color=selected_colors[i])

            if 'plot' in plot_interpolated and len(freq_new) > 0:
                ax.plot(freq_new, storage_moduli_interp, f'{marker}-', label=f'{plot_labels[i] if i < len(plot_labels) else test_index} G\' Interpolated', color=selected_colors[i])
                ax.plot(freq_new, loss_moduli_interp, f'{marker}-', label=f'{plot_labels[i] if i < len(plot_labels) else test_index} G\" Interpolated', color=selected_colors[i])

            if 'find' in find_g_equality and len(freq_new) > 0:
                diff = np.abs(storage_moduli_interp - loss_moduli_interp)
                min_diff_index = np.argmin(diff)
                g_equality_freq = freq_new[min_diff_index]
                g_equality_value = storage_moduli_interp[min_diff_index]
               
                # Check if the crossover frequency is meaningful
                if diff[min_diff_index] < 1:  # You can adjust this threshold as needed
                    ax.plot(g_equality_freq, g_equality_value, marker=marker, markersize=marker_size, label=f'{plot_labels[i] if i < len(plot_labels) else test_index} Cross Over Point', color=selected_colors[i])
                    crossover_freq_output.append(html.P(f'{plot_labels[i] if i < len(plot_labels) else test_index}: G\'=G\" @ {g_equality_freq:.2f} rad/s'))

            if 'raw' in raw_data_toggle and freq_specific is not None:
                idx = np.abs(freq - freq_specific).argmin()
                raw_gp = storage_moduli.iloc[idx]
                raw_gpp = loss_moduli.iloc[idx]
                phase_shift_angle = np.degrees(np.arctan(loss_moduli.iloc[idx] / storage_moduli.iloc[idx]))
                loss_factor = loss_moduli.iloc[idx] / storage_moduli.iloc[idx]
                raw_data_output.append(html.Div([
                    html.P(f'{plot_labels[i] if i < len(plot_labels) else test_index}:'),
                    html.P(f'Frequency: {freq_specific} rad/s'),
                    html.P(f'G\': {raw_gp}'),
                    html.P(f'G\": {raw_gpp}'),
                    html.P(f'Phase Shift Angle: {phase_shift_angle} degrees'),
                    html.P(f'Loss Factor: {loss_factor}')
                ]))

        ax.set_xlabel('Angular Frequency, ω/rad$s^{-1}$', fontname=font_picker, fontsize=font_size)
        ax.set_ylabel('G\' & G\" /Pa', fontname=font_picker, fontsize=font_size)
        ax.set_xscale('log')
        ax.set_yscale('log')

        ax.yaxis.set_major_formatter('{:.2f}'.format)
        ax.xaxis.set_major_formatter('{:.2f}'.format)
        ax.legend(prop={'family': font_picker, 'size': font_size})
        if 'show' in gridlines_toggle:
            ax.grid(True)
            ax.grid(which='minor', color='#EEEEEE')
        else:
            ax.grid(False)

        # Ensure x-axis and y-axis solid lines with ticks
        ax.tick_params(axis='both', which='both', direction='in', top=True, right=True, labelsize=font_size)
        ax.spines['top'].set_linewidth(1.5)
        ax.spines['right'].set_linewidth(1.5)
        ax.spines['left'].set_linewidth(1.5)
        ax.spines['bottom'].set_linewidth(1.5)

        if x_min is not None and x_max is not None:
            ax.set_xlim([x_min, x_max])
        plt.close(fig)
        plt.clf()

        # Convert plot to PNG image
        output = io.BytesIO()
        FigureCanvas(fig).print_png(output)
        encoded_image = base64.b64encode(output.getvalue()).decode('utf-8')
        img_src = f'data:image/png;base64,{encoded_image}'

        return data, columns, html.Img(src=img_src), raw_data_output, crossover_freq_output, ""

    except Exception as e:
        error_message = f"An error occurred: {str(e)}"
        return [], [], '', '', '', error_message


# Run the app then tyoe this http://127.0.0.1:2235/ to run in browser

"""These lines are for running your app and are almost always the same"""
if __name__ == '__main__':
    app.run(port=2235)
